In [1]:
(version)

"7.7"

In [2]:
#lang iracket/lang #:require bhdl/splicing #:reader bhdl/splicing

In [3]:
(list (list 1 2 3) ..)

'(1 2 3)

In [4]:
(putenv "BHDL_LIBRARY_PATH" "~/bhdl/bhdl-footprints")
(getenv "BHDL_LIBRARY_PATH")

"~/bhdl/bhdl-footprints"

In [5]:
(require racket/list
         racket/format
         (prefix-in pict: pict))

In [6]:
(require bhdl)

DEBUG:  unrecognized shape:  SVGNODE
DEBUG:  unrecognized shape:  SVGNODE
DEBUG:  unrecognized shape:  SVGNODE
DEBUG:  unrecognized shape:  ARC
DEBUG:  unrecognized shape:  ARC
DEBUG:  unrecognized shape:  SVGNODE
DEBUG:  unrecognized shape:  SVGNODE
DEBUG:  unrecognized shape:  SVGNODE
DEBUG:  unrecognized shape:  ARC
DEBUG:  unrecognized shape:  ARC


In [7]:
(define (add-ghost-top p sep)
  "Add some space on top of picture"
  (vc-append (pict:ghost (pict:rectangle 1 sep)) p))

In [8]:
(define spreadboard
  (make-circuit
   ;; FIXME inset a little bit
   #:vars ([jumper-mini-nano-SCL (PinHeader 3)]  ;j3
           [jumper-mini-nano-SDA (PinHeader 3)]  ;j4
           [jumper-micro-SDA (PinHeader 3)]      ;j10
           [jumper-micro-SCL (PinHeader 3)]      ;j11
           [jumper-micro-mini-VCC (PinHeader 3)] ;J1
           [jumper-mkr-MOSI (PinHeader 3)]       ;J5
           [jumper-mkr-SCK (PinHeader 3)]        ;J6
           [jumper-mkr-SDA (PinHeader 3)]        ;J8
           [jumper-mkr-SCL (PinHeader 3)]        ;J9
           [uno1 (Arduino-Uno)]
           [uno2 (Arduino-Uno)]
           [uno3 (Arduino-Uno-ICSP)]
           [uno4 (Arduino-Uno-ICSP)]
           [micro (Arduino-Micro)]
           [mini (Arduino-Mini)]
           [nano (Arduino-Nano)]
           [mkr (Arduino-MKR)])
   ;; The bunch of connections across all Arduinos
   #:connect
   (*=
    ;; TODO TX RX RESET VIN AREF IOREF
    (uno1  [A0 A1 A2 A3 A4 A5 -  -  -  -  D2 D3 D4 D5 D6 D7 D8 D9 D10 D11 D12 D13 -   GND 3V3 5V])
    (uno2  [A0 A1 A2 A3 A4 A5 -  -  -  -  D2 D3 D4 D5 D6 D7 D8 D9 D10 D11 D12 D13 -   GND 3V3 5V])
    (uno3  [A0 A1 A2 A3 A4 A5 -  -  -  -  D2 D3 D4 D5 D6 D7 D8 D9 D10 D11 D12 D13 VCC GND 3V3 5V])
    (uno4  [A0 A1 A2 A3 A4 A5 -  -  -  -  D2 D3 D4 D5 D6 D7 D8 D9 D10 D11 D12 D13 VCC GND 3V3 5V])
    (micro [A0 A1 A2 A3 -  -  -  -  -  -  -  -  D4 D5 D6 D7 D8 D9 D10 -   -   -   -   GND -   -])
    (mini  [A0 A1 A2 A3 -  -  A6 A7 -  -  D2 D3 D4 D5 D6 D7 D8 D9 D10 D11 D12 D13 -   GND -   -])
    (nano  [A0 A1 A2 A3 -  -  A6 A7 D0 D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 D11 D12 D13 -   GND 3V3 5V])
    ;; FIXME mkr doesn't have A7, but has D14
    (mkr   [A0 A1 A2 A3 A4 A5 A6 -  D0 D1 D2 D3 D4 D5 D6 D7 -  -  D10 -   -   D13 -   GND 3V3 5V]))
   ;; TODO define some global signal for e.g. uno1.SDA
   #:connect (list (*= (jumper-mkr-SDA [1 2 3])
                       ([uno1.D11 mkr.D11 uno1.SDA]))
                   (*= (jumper-mkr-SCL [1 2 3])
                       ([uno1.D12 mkr.D12 uno1.SCL]))
                   (*= (jumper-mkr-MOSI [1 2 3])
                       ;; CAUTION only Uno with ICSP headers has the MOSI pin
                       ([uno3.D8 mkr.D8 uno3.MOSI]))
                   (*= (jumper-mkr-SCK [1 2 3])
                       ([uno3.D9 mkr.D9 uno3.SCK]))
                   (*= (jumper-micro-SDA [1 2 3])
                       ([uno1.D2 micro.D2 uno1.SDA]))
                   (*= (jumper-micro-SCL [1 2 3])
                       ([uno1.D3 micro.D3 uno1.SCL]))
                   ;; jumpter mini/nano
                   (*- jumper-mini-nano-SCL.2 mini.A5 nano.A5)
                   (*- jumper-mini-nano-SCL.1 uno1.A5)
                   (*- jumper-mini-nano-SCL.3 uno1.SCL)
                   (*- jumper-mini-nano-SDA.2 mini.A4 nano.A4)
                   (*- jumper-mini-nano-SDA.1 uno1.A4)
                   (*- jumper-mini-nano-SDA.3 uno1.SDA)
                   ;; jumper micro mini
                   (*- jumper-micro-mini-VCC.2 micro.VCC mini.VCC)
                   (*- jumper-micro-mini-VCC.1 uno1.3V3)
                   (*- jumper-micro-mini-VCC.3 uno1.5V)
                   )
   #:layout (let ([g1 (ct-superimpose
                       uno1
                       (vc-append
                        10
                        (add-ghost-top mini 50)
                        (vc-append (rotate jumper-mini-nano-SCL (/ pi 2))
                                   (rotate jumper-mini-nano-SDA (/ pi 2)))))]
                  [g2 (ct-superimpose
                       uno2
                       (vc-append
                        10
                        (add-ghost-top micro 50)
                        (vc-append (rotate jumper-micro-SDA (/ pi 2))
                                   (rotate jumper-micro-SCL (/ pi 2))))
                       (rotate jumper-micro-mini-VCC (/ pi 2)))]
                  [g3 (ct-superimpose
                       uno3
                       (hc-append 10
                                  (vc-append (rotate jumper-mkr-MOSI (/ pi 2))
                                             (rotate jumper-mkr-SCK (/ pi 2))
                                             (rotate jumper-mkr-SDA (/ pi 2))
                                             (rotate jumper-mkr-SCL (/ pi 2)))
                                  (add-ghost-top (rotate mkr pi) 50)))]
                  [g4 (ct-superimpose uno4 (add-ghost-top nano 50))])
              (inset (vc-append 
               (ht-append 20 g1 g2)
               (ht-append 20 g3 g4))
                     20 20 20 -50))))

"DEBUG: /var/tmp/PKGS-bhdl-lib-bhdl-private-library-io.rkt-106-15_16046767161604676716149"
"DEBUG: /var/tmp/PKGS-bhdl-lib-bhdl-private-library-io.rkt-106-15_16046767161604676716165"
"DEBUG: /var/tmp/PKGS-bhdl-lib-bhdl-private-library-io.rkt-106-15_16046767161604676716193"
"DEBUG: /var/tmp/PKGS-bhdl-lib-bhdl-private-library-io.rkt-106-15_16046767161604676716218"
"DEBUG: /var/tmp/PKGS-bhdl-lib-bhdl-private-library-io.rkt-106-15_16046767161604676716225"
"DEBUG: /var/tmp/PKGS-bhdl-lib-bhdl-private-library-io.rkt-106-15_16046767161604676716234"
"DEBUG: /var/tmp/PKGS-bhdl-lib-bhdl-private-library-io.rkt-106-15_16046767161604676716246"


In [9]:
(parameterize ([current-directory "./out/spreadboard"]
               [padding-general 2])
    (circuit-export spreadboard
                    #:auto-place #f
                    #:formats '(kicad pdf png)))

generating KiCAD PCB ..
"DEBUG: /var/tmp/PKGS-bhdl-lib-bhdl-private-library-io.rkt-137-15_16046767161604676716478"
"DEBUG: /var/tmp/PKGS-bhdl-lib-bhdl-private-library-io.rkt-137-15_16046767161604676716502"
"DEBUG: /var/tmp/PKGS-bhdl-lib-bhdl-private-library-io.rkt-137-15_16046767161604676716556"
"DEBUG: /var/tmp/PKGS-bhdl-lib-bhdl-private-library-io.rkt-137-15_16046767161604676716590"
"DEBUG: /var/tmp/PKGS-bhdl-lib-bhdl-private-library-io.rkt-137-15_16046767161604676716602"
"DEBUG: /var/tmp/PKGS-bhdl-lib-bhdl-private-library-io.rkt-137-15_16046767161604676716617"
"DEBUG: /var/tmp/PKGS-bhdl-lib-bhdl-private-library-io.rkt-137-15_16046767161604676716633"
link: /home/jovyan/bhdl/bhdl-test/out/spreadboard/out.kicad_pcb
DEBUG:  Number of conflicts: 0
generating pdf ..
link: /home/jovyan/bhdl/bhdl-test/out/spreadboard/out.pdf
link: /home/jovyan/bhdl/bhdl-test/out/spreadboard/out.png


(pict '(prog #<procedure:...rivate/utils.rkt:1274:8> 480.0) 391.31002041128716 480.0 480.0 0.0 (list (child (pict '(picture 516.8992649059583 634.0539066545798 (put 0.0 0.0 #3019=(picture 516.8992649059583 634.0539066545798 (put 0 0 #3015=(picture 516.8992649059583 634.0539066545798 (put 0 0 #3013=(picture 516.8992649059583 634.0539066545798 (put 0.0 0.0 #3001=(picture 516.8992649059583 634.0539066545798 (put 0 0 #2997=(picture 516.8992649059583 634.0539066545798 (put 0 0 #2995=(picture 516.8992649059583 634.0539066545798 (put 0.0 0.0 #2983=(picture 516.8992649059583 634.0539066545798 (put 0 0 #2979=(picture 516.8992649059583 634.0539066545798 (put 0 0 #2977=(picture 516.8992649059583 634.0539066545798 (put 0.0 0.0 #2965=(picture 516.8992649059583 634.0539066545798 (put 0 0 #2961=(picture 516.8992649059583 634.0539066545798 (put 0 0 #2959=(picture 516.8992649059583 634.0539066545798 (put 0.0 0.0 #2947=(picture 516.8992649059583 634.0539066545798 (put 0 0 #2943=(picture 516.8992649059583 634.0539066545798 (put 0 0 #2941=(picture 516.8992649059583 634.0539066545798 (put 0.0 0.0 #2929=(picture 516.8992649059583 634.0539066545798 (put 0 0 #2925=(picture 516.8992649059583 634.0539066545798 (put 0 0 #2923=(picture 516.8992649059583 634.0539066545798 (put 0.0 0.0 #2911=(picture 516.8992649059583 634.0539066545798 (put 0 0 #2907=(picture 516.8992649059583 634.0539066545798 (put 0 0 #2905=(picture 516.8992649059583 634.0539066545798 (put 0.0 0.0 #2893=(picture 516.8992649059583 634.0539066545798 (put 0 0 #2889=(picture 516.8992649059583 634.0539066545798 (put 0 0 #2887=(picture 516.8992649059583 634.0539066545798 (put 0.0 0.0 #2875=(picture 516.8992649059583 634.0539066545798 (put 0 0 #2871=(picture 516.8992649059583 634.0539066545798 (put 0 0 #2869=(picture 516.8992649059583 634.0539066545798 (put 0.0 0.0 #2857=(picture 516.8992649059583 634.0539066545798 (put 0 0 #2853=(picture 516.8992649059583 634.0539066545798 (put 0 0 #2851=(picture 516.8992649059583 634.0539066545798 (put 0.0 0.0 #2839=(picture 516.8992649059583 634.0539066545798 (put 0 0 #2835=(picture 516.8992649059583 634.0539066545798 (put 0 0 #2833=(picture 516.8992649059583 634.0539066545798 (put 0.0 0.0 #2821=(picture 516.8992649059583 634.0539066545798 (put 0 0 #2817=(picture 516.8992649059583 634.0539066545798 (put 0 0 #2815=(picture 516.8992649059583 634.0539066545798 (put 0.0 0.0 #2803=(picture 516.8992649059583 634.0539066545798 (put 0 0 #2799=(picture 516.8992649059583 634.0539066545798 (put 0 0 #2797=(picture 516.8992649059583 634.0539066545798 (put 0.0 0.0 #2785=(picture 516.8992649059583 634.0539066545798 (put 0 0 #2781=(picture 516.8992649059583 634.0539066545798 (put 0 0 #2779=(picture 516.8992649059583 634.0539066545798 (put 0.0 0.0 #2767=(picture 516.8992649059583 634.0539066545798 (put 0 0 #2763=(picture 516.8992649059583 634.0539066545798 (put 0 0 #2761=(picture 516.8992649059583 634.0539066545798 (put 0.0 0.0 #2749=(picture 516.8992649059583 634.0539066545798 (put 0 0 #2745=(picture 516.8992649059583 634.0539066545798 (put 0 0 #2743=(picture 516.8992649059583 634.0539066545798 (put 0.0 0.0 #2731=(picture 516.8992649059583 634.0539066545798 (put 0 0 #2727=(picture 516.8992649059583 634.0539066545798 (put 0 0 #2725=(picture 516.8992649059583 634.0539066545798 (put 0.0 0.0 #2713=(picture 516.8992649059583 634.0539066545798 (put 0 0 #2709=(picture 516.8992649059583 634.0539066545798 (put 0 0 #2707=(picture 516.8992649059583 634.0539066545798 (put 0.0 0.0 #2695=(picture 516.8992649059583 634.0539066545798 (put 0 0 #2691=(picture 516.8992649059583 634.0539066545798 (put 0 0 #2689=(picture 516.8992649059583 634.0539066545798 (put 0.0 0.0 #2677=(picture 516.8992649059583 634.0539066545798 (put 0 0 #2673=(picture 516.8992649059583 634.0539066545798 (put 0 0 #2671=(picture 516.8992649059583 634.0539066545798 (put 0.0 0.0 #2659=(picture 516.8992649059583 634.0539066545798 (put 0 0 #2655=(picture 516.8992649059583 634.0539066545798 (put 0 0 #2653=(picture 516.89926490